In [1]:
from stock_monitor_helper import *
import pandas as pd
from pandas import DataFrame,Series
import tushare as ts
ts.set_token('61bd0479a5591b9a4c0112b0255ca10d29aeb1bfc2ea72cec0314725')
pro = ts.pro_api('61bd0479a5591b9a4c0112b0255ca10d29aeb1bfc2ea72cec0314725')
from datetime import datetime,timedelta
from collections import ChainMap
%load_ext autoreload
%autoreload 2

In [19]:
holding_stocks = pd.read_excel('stephen_stocks.xlsx')
all_code = dict(holding_stocks.set_index('Code')['Name'])

In [22]:
today_str = (datetime.now()- pd.tseries.offsets.DateOffset(days=1)).strftime("%Y%m%d")
pro = ts.pro_api()
Price = pro.daily(ts_code=','.join(all_code.keys()), adj='qfq',
                  start_date=today_str, fields='ts_code,trade_date,close')
# print(Price['trade_date'].max(), Price['trade_date'].min())
Price = Price.sort_values(['ts_code', 'trade_date'])
# Price['TodayPct'] = Price['close'] / Price['pre_close'] - 1
Price = Price.groupby(['ts_code']).aggregate({'close':'last'}).reset_index()
Price.rename(columns={'ts_code':'Code', 'close':'Price'}, inplace=True)

20210514 20210514


In [35]:
df = pd.merge(holding_stocks, Price, on='Code')
PRICE = dict(df.set_index('Name')['Price'].map(float))
# df

In [47]:
def change_stock(stock1, stock2, threshold=5):
	tmp = '\t%s:%s %s:%s ' %(stock1, stock2, round(PRICE[stock1],2), round(PRICE[stock2],2))
	if PRICE[stock1] > PRICE[stock2] + threshold:
		tmp += '可交易 ！！！！！'
	else:
		tmp += 'waiting...'
	print()
	print(tmp)

def buy_stock(stock, threshold):
	tmp = '\t%s %s ' %(stock, round(threshold,2))
	if PRICE[stock] < threshold:
		tmp += '可买入 ！！！！！'
	else:
		tmp += 'waiting...'
	print()
	print(tmp)

def sell_stock(stock, threshold):
	tmp = '\t%s %s ' %(stock, round(threshold,2))
	if PRICE[stock] > threshold:
		tmp += '可卖出 ！！！！！'
	else:
		tmp += 'waiting...'
	print()
	print(tmp)

In [54]:
## 换股为主观臆测，用于新进股票
print('换股:')
change_stock('泰格医药','药明康德',20)
change_stock('比亚迪','公牛集团',15)
change_stock('海螺水泥','东方雨虹',5) # 2/3
change_stock('海螺水泥','良品铺子',5) # 1/2
change_stock('格力电器','东方雨虹',15)
change_stock('格力电器','良品铺子',15)

换股:

	泰格医药:药明康德 156.35:153.2 waiting...

	比亚迪:公牛集团 153.53:182.11 waiting...

	海螺水泥:东方雨虹 49.83:55.62 waiting...

	海螺水泥:良品铺子 49.83:49.84 waiting...

	格力电器:东方雨虹 57.74:55.62 waiting...

	格力电器:良品铺子 57.74:49.84 waiting...


In [55]:
## 卖出 -- 破上次新高+10%
print('卖出:')
sell_stock('凯莱英', 337*1.1)
sell_stock('健帆生物', 92.63*1.1)

卖出:

	凯莱英 370.7 waiting...

	健帆生物 101.89 waiting...


In [56]:
## 买入 -- 高点回落20%
print('买入:')
# buy_stock('', '')

买入:
